# Add structured data



## Setup

In [1]:
%run 'shared.ipynb'

Connecting to Neo4j at neo4j://localhost:7687 as neo4j
Using data from /Users/akollegger/Developer/akollegger/genai/sec-edgar-notebooks/data/sample
Embedding with openai using text-embedding-ada-002
Chatting with openai using gpt-3.5-turbo


# Form 13 data in a csv

### Script - what is in the CSV

SEC Form 13 is filed by institutional investment management firms
to report what public companies the have invested in.

The forms are available as XML files. During data prepartion,
particular fields were pulled out of the XML and added as a
row in a CSV file.

These forms have been selected for investors of the public companies
that you previously imported from Form 10-K filings.


# Inspect Form 13 sources


### Script - read the csv into a list of records

To begin, you can read the csv file in using a `csv.DictReader`
which will parse each row and turn it into a dictionary
using the csv header row for keys.

In [2]:
import csv

all_form13s = []

with open(f'{DATA_DIR}/form13.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader: # each row will be a dictionary
      all_form13s.append(row)

first_form13 = all_form13s[0]

first_form13

{'source': 'https://sec.gov/Archives/edgar/data/1000097/0001000097-23-000009.txt',
 'managerCik': '1000097',
 'managerAddress': '152 WEST 57TH STREET, 50TH FLOOR, NEW YORK, NY, 10019',
 'managerName': 'KINGDON CAPITAL MANAGEMENT, L.L.C.',
 'reportCalendarOrQuarter': '2023-06-30',
 'cusip6': '697435',
 'cusip': '697435105',
 'companyName': 'PALO ALTO NETWORKS INC',
 'value': '27595080000.0',
 'shares': '108000'}

### script - look at more rows

You will start with the first row, then process all rows in the csv.

Let's take a quick look at what the rows look like. Maybe the first 5.

You can see that each of these firms have invested in the same company,
'netapp'

### Script - what are those fields?

You can see the following fields:

Details about the firm itself:
- managerName - the name of the firm
- managerCik - the unique "central index key" for the firm
- managerAddress - the business address of the firm

Then information about a particular investment:
- reportCalendarOrQuarter - the date reporting date
- companyName - the name of the public company they invested in
- value - the value of the investment
- shares - the number of shares of the investment

And some metadata about the company they invested in:
- cusip6 - the 6-digit cusip code
- cusip - the full cusip code 

All that becomes a row in the CSV file,
along with an extra column called "source"


# Form13 Node types

From each row, you will create two nodes: one for the management company 
and one for the company that they invest in.

The management company nodes will have a `:Manager` label.
They're unique based on the "central index key" from the SEC, 
which will be stored in the `managerCik` property.
They'll also get a `managerName` property.

The company nodes will have a `:Company` label.
They are unique based on a `cusip6` identifier.
The company nodes will also get a `companyName` and full `cusip` property from the form 13 data.

(possible slide)

### Script - Company nodes

Start with the company nodes.

You will merge a node labeled as `:Company` that is unique
by the `cusip6` identifier.

On creation, set the company name and the full `cusip` array of identifiers. 



### Script - create a constraint for company nodes

To ensure each company is unique and to speed up look-ups,
you can create a uniqueness constraint.

The cosntraint will apple to Nodes with the label `Company`,
requiring that `cusip6` identifier is unique.

In [3]:
# Create a uniqueness constraint on the managerCik property of Manager nodes 
gdb.execute_query("""
    CREATE CONSTRAINT unique_company 
        IF NOT EXISTS FOR (n:Company) 
        REQUIRE n.cusip6 IS UNIQUE
""")


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x134557350>, keys=[])

### Script - create all companies

To create all companies in the CSV file, we'll loop through the rows
and run that same `MERGE` query.

Using the `MERGE` will only create 1 company for every unique `cusip6` identifier,
no matter how many rows in the CSV contain information about that company.

In [4]:
cypher = """
MERGE (com:Company {cusip6: $form13Param.cusip6})
  ON CREATE
    SET com.name = $form13Param.companyName,
        com.cusip = $form13Param.cusip
"""

for form13 in all_form13s:
  gdb.execute_query(cypher, 
      form13Param = form13 
  )


In [6]:
gdb.execute_query("MATCH (com:Company) RETURN count(com)").records

[<Record count(com)=10>]

### Script - enrich the company node

The names from the Form 10K can be used to enrich the Company nodes.

Having a relationship in place means that navigating from a company to a
form is easy and fast, regardless of the overall size of the database.


In [7]:
# Create a full-text index of Manager names
gdb.execute_query("""
CREATE FULLTEXT INDEX fullTextCompanyNames
  IF NOT EXISTS
  FOR (com:Company) 
  ON EACH [com.names]
""")


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x134565990>, keys=[])

In [8]:
# Connect all `Company` nodes to their corresponding `Form` nodes
# based on the `cusip6` property

# MATCH a double node pattern, for the `Company` and `Form` nodes
# WHERE the `Company` and `Form` nodes have the same `cusip6` property
# MERGE to connect these pairs with a (:Company)-[:FILED]->(:Form) relationship
# RETURN a count of the number of relationships created or found (merged)
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  SET com.names = form.names
"""

gdb.execute_query(cypher)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x13386bc10>, keys=[])

### Script - connect companies to their forms

Using the same pair matching approach, you can connect
each Company to the Form 10k they filed with a `FILED` relationship.

Match the pairs of company and form, then `MERGE` the new relationship.

In [9]:
# Connect all `Company` nodes to their corresponding `Form` nodes
# based on the `cusip6` property

# MATCH a paired node pattern, for the `Company` and `Form` nodes
# WHERE the `Company` and `Form` nodes have the same `cusip6` property
# MERGE to connect these pairs with a (:Company)-[:FILED]->(:Form) relationship

gdb.execute_query("""
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  MERGE (com)-[:FILED]->(form)
""")


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x134561a10>, keys=[])

## Script - Manager nodes

The institutional investment managers nodes will have a `:Manager` label.

They're unique based on the "central index key" from the SEC, 
which will be stored in the `managerCik` property.
They'll also get a `managerName` property.

The manager information will get passed in as a query parameter
named `managerParam` that will have the first_form13 dictionary.


### Script - index the Manager nodes

There will be many management companies, so you can
create a uniqueness constraint to avoid creating duplicates.

Create a constraint on nodes with a `Manager` label,
requiring that the `managerCik` is unique.

In [10]:
# Create a uniqueness constraint on the managerCik property of Manager nodes 
gdb.execute_query("""
CREATE CONSTRAINT unique_manager 
  IF NOT EXISTS
  FOR (n:Manager) 
  REQUIRE n.cik IS UNIQUE
""")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x134560590>, keys=[])

*Script*: Also create a full text index on the manager nodes.

The full-text index is useful for keyword search.

A vector index allows searching based on similar concepts.

A full-text index allows searching based on similar looking strings.



In [11]:
# Create a full-text index of Manager names
gdb.execute_query("""
CREATE FULLTEXT INDEX fullTextManagerNames
  IF NOT EXISTS
  FOR (mgr:Manager) 
  ON EACH [mgr.name]
""")


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x134560a50>, keys=[])

### Script - create all management companies 

Now you will create nodes for all management firms 
that appear in the CSV file. Each of these firms
have made an investment in NetApp. 

Use python to loop through the rows.

This query is expecting the `managerParam` to be 
a dictionary, so you can pass in the entire row
as a dictionary.

When the cypher query runs, it will look for
fields from that dictionary to set properties 
on the nodes.


In [13]:
cypher = """
  MERGE (mgr:Manager {cik: $managerParam.managerCik})
    ON CREATE
        SET mgr.name = $managerParam.managerName,
            mgr.address = $managerParam.managerAddress
"""

for form13 in all_form13s:
  gdb.execute_query(cypher, managerParam=form13)


## Script - management investment relationships

We can now find pairs of Manager nodes and 
Company nodes using information from the form 13 csv.

Each row in the CSV represents a single investment.

Use the `cik` field to find a manager
and the `cusip6` field to find the company.

Here, we're looking for the investment described by the first form.

### Script - connect management firm with company

You can find a manager node amd the company they invested in.

You can now connect those nodes together. This is something
you've done before in the course. The query will get a little
long so let's go through it one line at a time.

...

using a `MERGE` clause.

Because a manager may report multiple investments in a company,
each investment is unique based on the reported calendar or quarter date.

The `MERGE` clause for the relationship will be unique for a 
particular pair of Manger and company nodes, plus the reported investment date.


### Script - connect all investments

You can now loop through all rows of the CSV file to
create a `OWNS_STOCK_IN` relationship between
management firms and companies they invested in.

In [14]:
cypher = """
  MATCH (mgr:Manager {cik: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
  MERGE (mgr)-[owns:OWNS_STOCK_IN { reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter }]->(com)
    ON CREATE
      SET owns.value  = toFloat($ownsParam.value), 
          owns.shares = toInteger($ownsParam.shares)
"""

for form13 in all_form13s:
  gdb.execute_query(cypher, ownsParam=form13)


### Script - check the number of investments

You can check how many investments were created
by counting the number of `OWNS_STOCK_IN` relationships.

MATCH the pattern from Manager OWNS_STOCK_IN Company.

Then return a count of the relationships.

OK, there are just as many relationships as we had
rows in the CSV. Perfect.

In [15]:
cypher = """
  MATCH (:Manager)-[owns:OWNS_STOCK_IN]->(:Company)
  RETURN count(owns) as investments
"""

gdb.execute_query(cypher)


EagerResult(records=[<Record investments=6172>], summary=<neo4j._work.summary.ResultSummary object at 0x134615710>, keys=['investments'])

# Example Cypher queries

### script - why did we do all that?

You've added structured data to the knowledge graph,
connected to the unstructured data from text in the forms.

This let's you ask some questions that are not possible
to answer by just looking at the text. 

Let's look at the some patterns that provide extra context
for information discovery questions.

### Script - well known chunk 

What I love about graphs, is that they are awesome for exploration.

Let's have some fun looking around to see what we can find.

...

To begin, find an a random chunk to use in later queries.

You can match all chunks, then return the `chunkId` from the first one.

Save that value in a python variable called `well_known_chunk` by 
grabbing the first result row, then the `chunkId` field in that row.

In [16]:
cypher = """
    MATCH (chunk:Chunk)
    RETURN chunk.chunkId as chunkId LIMIT 1
    """

chunk_rows = gdb.execute_query(cypher).records

print(chunk_rows)

chunk_first_row = chunk_rows[0]

print(chunk_first_row)

ref_chunk_id = chunk_first_row['chunkId']

ref_chunk_id


[<Record chunkId='0001650372-23-000040-item1-chunk0000'>]
<Record chunkId='0001650372-23-000040-item1-chunk0000'>


'0001650372-23-000040-item1-chunk0000'

In [17]:
gdb.execute_query("""
    MATCH (chunk:Chunk { chunkId: $chunkIdParam })
    RETURN chunk.chunkId as chunkId LIMIT 1
""", chunkIdParam=ref_chunk_id).records


[<Record chunkId='0001650372-23-000040-item1-chunk0000'>]

### Script - find a form from a chunk

You can work backwards from the chunk to find
the Form that is a part of.

Then return the source link for the Form.

In [19]:
# Retrieve a form, working backwards from a specific Chunk...

# MATCH a 2 node pattern, anchored by a specified `Chunk` 
# that is `PART_OF` a `Form`.
# RETURN the `companyName` property of the `Company` 
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form)
    RETURN f.source
    """

gdb.execute_query(cypher, chunkIdParam=ref_chunk_id).records

[<Record f.source='https://www.sec.gov/Archives/edgar/data/1650372/000165037223000040/0001650372-23-000040-index.htm'>]

### Script - find a company from a chunk

You can extend the pattern to find the company that filed the form.

Start with the same pattern, from Chunk to Form,
then use a reverse relationship from the Form to 
the company that filed that form.

In [25]:
# Retrieve a company, working backwards from a specific Chunk...

# MATCH a 3 node pattern, anchored by a specified `Chunk` 
# that is `PART_OF` a `Form` which was `FILED` by a `Company`.
# RETURN the `companyName` property of the `Company` 
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
    (com:Company)-[:FILED]->(f)
RETURN com.name as name
"""

gdb.execute_query(cypher, chunkIdParam=ref_chunk_id).records

[<Record name='ATLASSIAN CORP PLC'>]

### Script - investors in a company, starting from a chunk

Keep extending the pattern all the way to the investment managers.

This pattern goes from Chunk to Form,
to Company that filed that Form,
and then to Managers that own stock in the Company.

Since this pattern is long, we can split it up into 
two patterns. The first makes it the Form, stored in
the variable `f`. 

Then, that same `f` is used in the second pattern,
uniting both patterns. The variable `f` must be the 
same in both patterns for a match to be made.

Return the company and a count of the management firms
that invested in the company.

...

In this single Company dataset, 
all the managers have invested in "Netapp".


In [26]:
# Retrieve investors of a company, working backwards from a specific Chunk...

# MATCH a 4 node pattern, anchored by a specified `Chunk` 
# that is `PART_OF` a `Form` which was `FILED` by a `Company`
# which a `Manager` `OWNS_STOCK_IN`.
# RETURN the company name along with a count of the management firm names
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[:OWNS_STOCK_IN]->(com)
RETURN com.name, 
        count(mgr.name) as numberOfinvestors 
LIMIT 1
"""

gdb.execute_query(cypher, chunkIdParam=ref_chunk_id).records

[<Record com.name='ATLASSIAN CORP PLC' numberOfinvestors=8>]

### script - graph powah!

You're starting to see what is possible with a knowledge graph,
getting information that wouldn't be possible with a vector search.


### Script - investment as a sentence

That pattern you just created, from a chunk
all the way to an investor, is useful information.

You can use that information for expanding the
context provided to an LLM. You can turn the
data into a string that looks like sentences
that an LLM will happily understand.

For example, you can find investors for a company,
then create sentences that contain details
about each investment
to further extend the information provided to
an LLM.

Use the same MATCH pattern as before

then return a string by adding together some
of the fields along with some string literals.

This uses two functions to nicely format the value
of the investment.

First, `toInteger()` is used to convert to an integer value.

Then the `apoc.number.format()` function adds commas to 
improve readability.

In [27]:
# Retrieve investors of a company, working backwards from a specific Chunk...

# MATCH a 4 node pattern, anchored by a specified `Chunk` 
# that is `PART_OF` a `Form` which was `FILED` by a `Company`
# which a `Manager` `OWNS_STOCK_IN`.
# RETURN a sentence about the manager's investment in the company 
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
    RETURN mgr.name + " owns " + owns.shares + 
        " shares of " + com.name + 
        " at a value of $" + 
        apoc.number.format(toInteger(owns.value)) AS text
    LIMIT 10
    """

gdb.execute_query(cypher, chunkIdParam=ref_chunk_id).records

[<Record text='HUNTINGTON NATIONAL BANK owns 1 shares of ATLASSIAN CORP PLC at a value of $211,000'>,
 <Record text='Investment Research & Advisory Group, Inc. owns 45 shares of ATLASSIAN CORP PLC at a value of $7,551,000'>,
 <Record text='Retirement Group, LLC owns 2 shares of ATLASSIAN CORP PLC at a value of $336,000'>,
 <Record text='Harvest Fund Management Co., Ltd owns 746 shares of ATLASSIAN CORP PLC at a value of $125,000'>,
 <Record text='Kopp Family Office, LLC owns 49670 shares of ATLASSIAN CORP PLC at a value of $8,335,000'>,
 <Record text='Stanley-Laman Group, Ltd. owns 45188 shares of ATLASSIAN CORP PLC at a value of $7,582,998,000'>,
 <Record text='AFT, FORSYTH & COMPANY, INC. owns 5135 shares of ATLASSIAN CORP PLC at a value of $861,704,000'>,
 <Record text='Insight Holdings Group, LLC owns 91800 shares of ATLASSIAN CORP PLC at a value of $15,404,958,000'>]

## Example RAG questions

### Script - ready for RAG

OK, you see the kind of things you can do with pattern matching.

Next, you can put that to use in a RAG workflow.

## Prepare langchain for querying the Knowledge Graph



### Script - vector based chain

Now create a question and answer chain that
uses the default similarity search without
any extra information.

Call this the `plain_chain`.

In [29]:


# Create a langchain vector store from the existing Neo4j knowledge graph.
vector_store = Neo4jVector.from_existing_graph(
    embedding=embeddings_api,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
plain_chain = prettifyChain(RetrievalQAWithSourcesChain.from_chain_type(
    chat_api, 
    chain_type="stuff", 
    retriever=retriever
))

# Script - retrieval query



### Script - chain with extra information

Now create another chain that extends the retrieval
query to add investment information.

The retrieval query does a bit extra work, collecting
the top-10 investment statements into a list,
then joinging that list into a single string
that then gets preprepended to the text from the
vector similarity search.

Recall that the `node` is the Chunk found by vector search,
and the `score` variable is the similarity score of that Chunk.

In [30]:
investment_retrieval_query = """
MATCH (node)-[:PART_OF]->(f:Form),
    (f)<-[:FILED]-(com:Company),
    (com)<-[owns:OWNS_STOCK_IN]-(mgr:Manager)
WITH node, score, mgr, owns, com 
    ORDER BY owns.shares DESC LIMIT 10
WITH collect (
    mgr.name + 
    " owns " + owns.shares + " of " + com.name + 
    " at a value of $" + apoc.number.format(owns.value) + "." 
) AS investment_statements, node, score
RETURN apoc.text.join(investment_statements, "\n") + 
    "\n" + node.text AS text,
    score,
    { 
      source: node.source
    } as metadata
"""

vector_store_with_investment = Neo4jVector.from_existing_index(
    embedding=embeddings_api,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=investment_retrieval_query,
)

# Create a retriever from the vector store
retriever_with_investments = vector_store_with_investment.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
investment_chain = prettifyChain(RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever_with_investments
))

## Example user questions

In [31]:
question = 'In a single sentence, tell me about Netapp.'

In [34]:
question = 'In a single sentence, tell me about Netapp investors.'

In [35]:
plain_chain(question)

Netapp investors are individuals or entities that are interested in cloud-led,
data-centric software companies, with a focus on enterprise storage, data
management, and cloud operations markets, fostering partnerships and global
relationships for success.


In [36]:
investment_chain(question)

VANGUARD GROUP INC, BlackRock Inc., and PRIMECAP MANAGEMENT CO/CA/ are investors
in NetApp Inc.
